# 1. Work with data

work with data and output to a new data set

In [ ]:
base_path = "C:\\Users\\hasee\\workspace\\Kaggle\\safe_driver\\" # your folder

import pandas as pd
train=pd.read_csv(base_path + 'train.csv')
test=pd.read_csv(base_path + 'test.csv')

train['ps_ind_0609_bin'] = train.apply(lambda x: 1 if x['ps_ind_06_bin'] == 1 else (2 if x['ps_ind_07_bin'] == 1 else 
(
3 if x['ps_ind_08_bin'] == 1 else (4 if x['ps_ind_09_bin'] == 1 else 5)

)), axis = 1)

test['ps_ind_0609_bin'] = test.apply(lambda x: 1 if x['ps_ind_06_bin'] == 1 else (2 if x['ps_ind_07_bin'] == 1 else 
(
3 if x['ps_ind_08_bin'] == 1 else (4 if x['ps_ind_09_bin'] == 1 else 5)

)), axis = 1)

train.drop(['ps_ind_06_bin', 'ps_ind_07_bin', 'ps_ind_08_bin', 'ps_ind_09_bin'], axis = 1, inplace = True)

test.drop(['ps_ind_06_bin', 'ps_ind_07_bin', 'ps_ind_08_bin', 'ps_ind_09_bin'], axis = 1, inplace = True)

train['ps_car_13'] = (train['ps_car_13']*train['ps_car_13']* 48400).round(0)

test['ps_car_13'] = (test['ps_car_13']*test['ps_car_13']* 48400).round(0)

train['ps_car_12'] = (train['ps_car_12']*train['ps_car_12']).round(4) * 10000

test['ps_car_12'] = (test['ps_car_12']*test['ps_car_12']).round(4) * 10000

for c in train[[c for c in train.columns if 'bin' in c]].columns:
    for cc in train[[c for c in train.columns if 'bin' in c]].columns:
            if train[train[cc] * train[c] == 0].shape[0] == train.shape[0]:
                print(c, cc)

train['ps_ind_161718_bin'] = train.apply(lambda x: 1 if x['ps_ind_16_bin'] == 1 else
                                        (2 if x['ps_ind_17_bin'] == 1 else 3), axis = 1
                                        )

test['ps_ind_161718_bin'] = test.apply(lambda x: 1 if x['ps_ind_16_bin'] == 1 else
                                        (2 if x['ps_ind_17_bin'] == 1 else 3), axis = 1
                                        )

train.drop(['ps_ind_16_bin', 'ps_ind_17_bin', 'ps_ind_18_bin'], axis = 1, inplace = True)

test.drop(['ps_ind_16_bin', 'ps_ind_17_bin', 'ps_ind_18_bin'], axis = 1, inplace = True)

train.to_csv(base_path + 'train_p.csv', index = False)

test.to_csv(base_path + 'test_p.csv', index = False)

ps_ind_16_bin ps_ind_17_bin
ps_ind_16_bin ps_ind_18_bin
ps_ind_17_bin ps_ind_16_bin
ps_ind_17_bin ps_ind_18_bin
ps_ind_18_bin ps_ind_16_bin
ps_ind_18_bin ps_ind_17_bin


In [3]:
train

id  target  ps_ind_01  ps_ind_02_cat  ps_ind_03  ps_ind_04_cat  \
0             7       0          2              2          5              1   
1             9       0          1              1          7              0   
2            13       0          5              4          9              1   
3            16       0          0              1          2              0   
4            17       0          0              2          0              1   
5            19       0          5              1          4              0   
6            20       0          2              1          3              1   
7            22       0          5              1          4              0   
8            26       0          5              1          3              1   
9            28       1          1              1          2              0   
10           34       0          5              2          2              0   
11           35       0          2              1          3              1   
12           36       0          2              1          1              1   
13           43       0          1              1          3              1   
14           46       0          5              1         11              0   
15           48       0          5              1          3              1   
16           50       0          1              2          1              0   
17           58       0          5              1          6              0   
18           61       0          5              1          8              0   
19           64       1          0              1          2              1   
20           65       0          0              1          2              0   
21           66       0          0              1          5              1   
22           72       0          5              3          6              1   
23           74       0          2              1          2              1   
24           77       0          0              1          2              0   
25           78       0          0              1          7              0   
26           79       0          0              1          4              1   
27           80       0          4              1          6              0   
28           84       1          0              2          0              1   
29           85       0          3              2          2              0   
...         ...     ...        ...            ...        ...            ...   
595182  1487945       0          0              1          2              0   
595183  1487951       0          1              1          9              0   
595184  1487952       0          1              1          5              0   
595185  1487954       0          0              2          8              1   
595186  1487957       0          0              1          3              1   
595187  1487958       0          0              1          7              0   
595188  1487962       0          2              1          3              0   
595189  1487963       0          3              1          9              0   
595190  1487964       0          1              2          3              0   
595191  1487968       0          5              1          3              0   
595192  1487973       0          0              1          1              0   
595193  1487975       0          0              2          0              0   
595194  1487976       0          0              1          7              1   
595195  1487980       0          0              3          3              1   
595196  1487983       0          1              1          2              0   
595197  1487988       0          1              1          7              1   
595198  1487990       0          5              1          7              1   
595199  1487992       0          3              1          3              1   
595200  1487994       0          1              1          1   

In [5]:
from scipy.stats import reciprocal, uniform
reciprocal(0.01,0.1)

# 2. XGBOOST and LGB from kaggle kernel 

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
import gc

print('loading files...')
train = pd.read_csv(base_path+'train_p.csv', na_values=-1)
test = pd.read_csv(base_path+'test_p.csv', na_values=-1)
col_to_drop = train.columns[train.columns.str.startswith('ps_calc_')]
train = train.drop(col_to_drop, axis=1)  
test = test.drop(col_to_drop, axis=1)  

for c in train.select_dtypes(include=['float64']).columns:
    train[c]=train[c].astype(np.float32)
    test[c]=test[c].astype(np.float32)
for c in train.select_dtypes(include=['int64']).columns[2:]:
    train[c]=train[c].astype(np.int8)
    test[c]=test[c].astype(np.int8)    

print(train.shape, test.shape)

# custom objective function (similar to auc)

def gini(y, pred):
    g = np.asarray(np.c_[y, pred, np.arange(len(y)) ], dtype=np.float)
    g = g[np.lexsort((g[:,2], -1*g[:,1]))]
    gs = g[:,0].cumsum().sum() / g[:,0].sum()
    gs -= (len(y) + 1) / 2.
    return gs / len(y)

def gini_xgb(pred, y):
    y = y.get_label()
    return 'gini', gini(y, pred) / gini(y, y)

def gini_lgb(preds, dtrain):
    y = list(dtrain.get_label())
    score = gini(y, preds) / gini(y, y)
    return 'gini', score, True

# xgb
params = {'eta': 0.02, 'max_depth': 4, 'subsample': 0.9, 'colsample_bytree': 0.9, 
          'objective': 'binary:logistic', 'eval_metric': 'auc', 'silent': True}

X = train.drop(['id', 'target'], axis=1)
features = X.columns
X = X.values
y = train['target'].values
sub=test['id'].to_frame()
sub['target']=0

sub_train = train['id'].to_frame()
sub_train['target']=0

nrounds=10**6  # need to change to 2000
kfold = 5  # need to change to 5
skf = StratifiedKFold(n_splits=kfold, random_state=0)
for i, (train_index, test_index) in enumerate(skf.split(X, y)):
    print(' xgb kfold: {}  of  {} : '.format(i+1, kfold))
    X_train, X_valid = X[train_index], X[test_index]
    y_train, y_valid = y[train_index], y[test_index]
    d_train = xgb.DMatrix(X_train, y_train) 
    d_valid = xgb.DMatrix(X_valid, y_valid) 
    watchlist = [(d_train, 'train'), (d_valid, 'valid')]
    xgb_model = xgb.train(params, d_train, nrounds, watchlist, early_stopping_rounds=100, 
                          feval=gini_xgb, maximize=True, verbose_eval=100)
    sub['target'] += xgb_model.predict(xgb.DMatrix(test[features].values), 
                        ntree_limit=xgb_model.best_ntree_limit+50) / (kfold)
    
    sub_train['target'] += xgb_model.predict(xgb.DMatrix(train[features].values), 
                        ntree_limit=xgb_model.best_ntree_limit+50) / (kfold)
    
gc.collect()
sub.head(2)

sub.to_csv(base_path+'test_sub_xgb.csv', index=False, float_format='%.5f')
sub_train.to_csv(base_path+'train_sub_xgb.csv', index=False, float_format='%.5f')


# lgb
sub['target']=0
sub_train['target']=0

params = {'metric': 'auc', 'learning_rate' : 0.01, 'max_depth':8, 'max_bin':10,  'objective': 'binary', 
          'feature_fraction': 0.8,'bagging_fraction':0.9,'bagging_freq':5,  'min_data': 500}

skf = StratifiedKFold(n_splits=kfold, random_state=1)
for i, (train_index, test_index) in enumerate(skf.split(X, y)):
    print(' lgb kfold: {}  of  {} : '.format(i+1, kfold))
    X_train, X_eval = X[train_index], X[test_index]
    y_train, y_eval = y[train_index], y[test_index]
    lgb_model = lgb.train(params, lgb.Dataset(X_train, label=y_train), nrounds, 
                  lgb.Dataset(X_eval, label=y_eval), verbose_eval=100, 
                  feval=gini_lgb, early_stopping_rounds=100)
    sub['target'] += lgb_model.predict(test[features].values, 
                        num_iteration=lgb_model.best_iteration) / (kfold)
    sub_train['target'] += lgb_model.predict(train[features].values, 
                        num_iteration=lgb_model.best_iteration) / (kfold)
    
sub.to_csv(base_path+'test_sub_lgb.csv', index=False, float_format='%.5f') 
sub_train.to_csv(base_path+'train_sub_lgb.csv', index=False, float_format='%.5f')

gc.collect()
sub.head(2)

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


loading files...
(595212, 34) (892816, 33)
 xgb kfold: 1  of  5 : 
[0]	train-gini:0.197799	valid-gini:0.207655
Multiple eval metrics have been passed: 'valid-gini' will be used for early stopping.

Will train until valid-gini hasn't improved in 100 rounds.
[100]	train-gini:0.246897	valid-gini:0.240157
[200]	train-gini:0.275545	valid-gini:0.257312
[300]	train-gini:0.29678	valid-gini:0.269755
[400]	train-gini:0.310673	valid-gini:0.276489
[500]	train-gini:0.320901	valid-gini:0.278977
[600]	train-gini:0.329141	valid-gini:0.280097
[700]	train-gini:0.336405	valid-gini:0.280587
[800]	train-gini:0.343045	valid-gini:0.280984
Stopping. Best iteration:
[771]	train-gini:0.341095	valid-gini:0.281079

 xgb kfold: 2  of  5 : 
[0]	train-gini:0.198356	valid-gini:0.192375
Multiple eval metrics have been passed: 'valid-gini' will be used for early stopping.

Will train until valid-gini hasn't improved in 100 rounds.
